In [ ]:
"""
The work here should condense the process require to take an unstructured, or a file with multiple, inconsistent structures within it, and turn it into a 2d object, or Pandas DataFrame, 
which in turn is translated to into both a SQL - accessible interface from the terminal via DuckDB as well as visually via Datasette and related meta reports, so a user can quickly and 
painlessly find the issue in a huge messy stack of logs

1. 

In [111]:
import pandas as pd
import re
import sys

In [37]:
line = ''.join([
        '2024-04-09 20:30:55 ', 
        'source > ',
        'INFO ',
        'debezium-sqlserverconnector-facppm1-change-event-source-coordinator ', 
        ''.join([
            'i.d.c.s.SqlServerStreamingChangeEventSource(lambda$getChangeTablesToQuery$4):',
            '410 CDC is enabled for table Capture instance "dbo_CMC_PMDD_DLNQ_DEF" ',
        ]),
        ''.join(['[sourceTableId=facppm1.dbo.CMC_PMDD_DLNQ_DEF, ',
                 'changeTableId=facppm1.cdc.dbo_CMC_PMDD_DLNQ_DEF_CT, ',
                 'startLsn=000658b3:0058f37f:0004, ',
                 'changeTableObjectId=1642202092, ',
                 'stopLsn=NULL] ',
                ]),
        'but the table is not on connector\'s table include list'
        # Add more log lines as needed
    ])

In [85]:
extract = df['log_line'].str.extract(r'{}'.format(r'"([^"]+)"')) 
type(extract)


pandas.core.frame.DataFrame

In [116]:
# Sample series of log lines
data = {
    'log_line': [
        line,
        "2024-03-27 15:43:48 platform > Using default value for environment variable SOCAT_KUBE_CPU_LIMIT: '2.0'",
        "2024-03-09 11:03:26 destination > 2024-03-09 11:01:29 INFO i.a.c.i.d.s.S3ConsumerFactory(lambda$onStartFunction$1):102 - Preparing storage area in destination completed.",
        "2024-03-09 11:03:26 source > INFO main i.a.i.s.m.MssqlSource(main):660 starting source: class io.airbyte.integrations.source.mssql.MssqlSource",
        "2024-03-27 15:43:48 platform > source-declarative-manifest-check-99-0-atllj stdoutLocalPort = 9038",
]
        # Add more log lines as needed
}

# line below clears out current df from memory
if not df.empty: del df

df = pd.DataFrame(data)

# TODO delete these two lines
# print(df)
# sys.exit()

In [118]:
df['extracted'] = df['log_line'].str.extract(r'{}'.format(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} (\w+) >')) 
df['extracted']

0         source
1       platform
2    destination
3         source
4       platform
Name: extracted, dtype: object

In [ ]:

# Define the regex pattern for each column
patterns = {
    'Timestamp': r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})',
    'LogLevel': r'((?<=\> )INFO)',
    'LogMessage': 
    'ConnectorType': r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} (\w+) >',
    'SourceComponent': r'((?<=INFO )[\w-]+)',
    'TableName': r'"([^"]+)"',
    'SourceTableID': r'sourceTableId=([^,]+)',
    'ChangeTableID': r'changeTableId=([^,]+)',
    'StartLSN': r'startLsn=([^,]+)',
    'ChangeTableObjectID': r'changeTableObjectId=([^,]+)',
    'StopLSN': r'stopLsn=([^\]]+)'
}

# Apply regex patterns to extract each component
count = 0
for column, pattern in patterns.items():
    # if count <= 8:
    print(f"columns: {df.columns}")
    print(f"extracted value: {df['log_line'].str.extract(r'{}'.format(pattern))}")
    df[column] = df['log_line'].str.extract(r'{}'.format(pattern))  # r'{}'.format(s)
    # display(df)
    #     count += 1
    # else:
    #     break
# df.drop(columns=['log_line'], inplace=True)  # Optional: remove the original log_line column if not needed

print(df)


In [120]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

df.columns
df.loc[0]


log_line               2024-04-09 20:30:55 source > INFO debezium-sql...
extracted                                                         source
Timestamp                                            2024-04-09 20:30:55
LogLevel                                                            INFO
ConnectorType                                                     source
SourceComponent        debezium-sqlserverconnector-facppm1-change-eve...
TableName                                          dbo_CMC_PMDD_DLNQ_DEF
SourceTableID                              facppm1.dbo.CMC_PMDD_DLNQ_DEF
ChangeTableID                       facppm1.cdc.dbo_CMC_PMDD_DLNQ_DEF_CT
StartLSN                                          000658b3:0058f37f:0004
ChangeTableObjectID                                           1642202092
StopLSN                                                             NULL
Name: 0, dtype: object

In [ ]:
import re
import pandas as pd
from pandas import DataFrame
import pathlib

from typing import (
    Dict,
    Str,
    
)

In [ ]:
import os
from typing import Union, Optional, Literal
import pandas as pd

class ExtDataFrame(pd.DataFrame):
    def __init__(self, *args, **kwargs):
        super(ExtDataFrame,  self).__init__(*args, **kwargs)
        self.raw_file = Optional[str

    @property
    def _constructor(self):
        return ExtDataFrame

    def _find_matching_lines(line, line_number, current_entry, entry_line_numbers):
        if pattern.match(line):
            if current_entry:
                # Save the previous log entry and its line numbers
                log_entries.append((''.join(current_entry), entry_line_numbers))
                current_entry = []
                entry_line_numbers = []
            # Start a new log entry
            current_entry.append(line)
            entry_line_numbers.append(line_number)
        else:
            # Continue adding lines to the current log entry
            current_entry.append(line)
            entry_line_numbers.append(line_number)

        return entry_line_numbers, current_entry

    def _fetch_log_entries(file_path: Path, pattern: str):
        """
        """
        
        # Regex pattern to identify the start of a log entry (a line starting with a timestamp)
        # pattern = re.compile(r'^(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})', re.MULTILINE)
        
        log_entries = []
        current_entry = []
        entry_line_numbers = []
    
        with open(file_path, 'r') as file:
            for line_number, line in enumerate(file, start=1):
                entry_line_numbers, current_entry = _find_matching_lines(line, line_number, current_entry, entry_line_numbers)
            
            # Don't forget to add the last entry after the loop ends
            if current_entry:
                log_entries.append((''.join(current_entry), entry_line_numbers))
    
        return log_entries

    def read_unstructured_text(text: line_parse_rule: str,
    def read_unstructured_file(file_path: Union[str, os.PathLike] = None, column_parse_rule: Optional[Dict[str,str], method ): -> pd.DataFrame
        """This method can be used on a plain text file, or any raw, complex strings where no obvious or consistent 2d structure is present or 
        known parsing method is available. Instead, the user passes in a parse rule dictionary uses a dictionary with key values representing columns 
        and Python regex patterns as corresponding values is passed into the function.

        For each key-value pair, a column is defined and data matching the pattern is returned.

        Args:
            

        Returns:

        Raises:

        Example:
            >>> test_content = "2024-03-27 15:45:17 Some log entry\\nMore details follow.\\n2024-03-27 15:45:18 Another log entry"
            >>> read_log_with_line_numbers(test_content)
            [('Some log entry\\nMore details follow.', range(1, 3)), ('Another log entry', range(3, 4))]
        
        """

    def 

In [ ]:
# Example usage
file_path = '/Users/bdavis/bengineerdavis/sawmill/test_files/data_engineering_job_99_attempt_1_txt.txt'
parse_rules = 
# log_entries_with_line_numbers = read_log_with_line_numbers(file_path)
log_entries_with_line_numbers =  ExtDataFrame().read_unstructured(file_path, parse_rules: Dict[str,str])

for entry, line_numbers in log_entries_with_line_numbers:
    print("Log Entry Lines:", line_numbers)
    print("Log Entry Content:\n", entry)
    print("---")